In [1]:
from pyxdf import load_xdf
import numpy as np

In [2]:
data, header = load_xdf('egi.xdf', synchronize_clocks = False, dejitter_timestamps = True)

In [3]:
type_dict = {data[i]['info']['type'][0]: i for i in range(len(data))}
EEG = type_dict['EEG']
MARKER = type_dict['Markers']

In [4]:
ev = data[EEG]['time_series'][:,-1] == 65376. # event code
not_ev_prev = np.logical_not(ev[:-1])
ev = ev[1:]
new_ev = np.logical_and(ev, not_ev_prev) # only record timestamp of event start
new_ev = np.array([False] + new_ev.tolist())

hardware_events = data[EEG]['time_stamps'][new_ev]
software_events = data[MARKER]['time_stamps']

In [5]:
lags = []
for event in hardware_events:
    se = np.max(software_events[software_events < event])
    lags.append(event - se)

np.mean(lags) * 1e3 # mean offset in milliseconds

8.850954016860651

In [6]:
np.std(lags) * 1e3 # jitter in milliseconds

0.302083649368636